## **1. OBJECTIFS**

Nous voulons évaluer rigoureusement la robustesse de notre modélisation et notre implémentation au-delà de sa seule validité théorique et sa faisabilité technique. Dans quelles conditions notre approche produit-elle des solutions de bonne qualité en un temps de calcul raisonnable ? Avec quelle variabilité ? Comment se comporte l'algorithme au cours de sa recherche, et qu'est ce qui explique son succès où son échec ?

Cette étude vise ainsi à établir la viabilité de notre solution à travers une **évaluation empirique** complète, en se reposant sur des benchmarks standardisés, des analyses statistiques et une lecture comportementale. Elle soulève les questions suivantes :  
- Qualité : l'écart relatif à un optimum connu et le respect strict des contraintes
- Robustesse : la variabilité et la part de l'aléatoire entre différentes exécutions des algorithmes, ainsi que la sensibilité aux paramètres et aux changements
- Scalabilité : la façon dont le temps d'exécution et de l'écart à l'optimum évoluent lorsque la taille des données d'entrée et la difficulté des instances croissent.
- Comportement : la trajectoire et l'équilibre entre l'intensification et la diversification

Pour y répondre, nous mettrons en œuvre un **plan d'expériences** (Design of Experiments - **DoE**). Cette approche méthodologique permet d'étudier l'effet de multiples facteurs (les paramètres de notre algorithme, les données d'une instance) sur les indicateurs de performance. Plutôt que de faire varier un seul paramètre à la fois, un plan d'expériences nous permettra d'identifier efficacement les facteurs les plus influents, de quantifier leur impact et de déceler d'éventuelles interactions entre eux.

Afin de garantir toute reproductibilité et transparence, nous fourniront le plus de détail possible sur l'environnement utilisé, les graines d'aléatoire, les temps limites, les paramètres, les fichiers d'instances, les décisions prises ou encore les benchmarks.

## **2. PROTOCOLE**

#### **2.1. BENCHMARKS**

Les performances de notre algorithme sont évaluées sur un ensemble d'instances de référence, issus de trois **benchmarks** reconnus dans la littérature pour notre problème.

Un **benchmark** est un ensemble d'instances représentatives, pour lesquelles il existe à chaque fois une **meilleure solution connue** (Best Know Solution, **BKS**), c'est-à-dire la meilleure solution trouvée à ce jour par la communauté scientifique. Les BKS ne sont pas systématiquement les solutions optimales, mais elles permettent déjà de se situer, et ce par rapport aux meilleurs algorithmes existants.

Nous avons sélectionnés 3 benchmarks, tous reconnus dans la littérature pour notre problème et respectivement basés sur les instances issues de la bibliothèque **VRPLIB**, des instances **SOLOMON**, introduites par Solomon (1987), et des instances **HG**, introduites par Hommbeger et Gehring (1999).

#### **2.2. INDICATEURS DE PERFORMANCE**

Pour évaluer les résultats de notre algorithme, nous nous baserons sur plusieurs indicateurs.
Pour mesurer la qualité, nous prendrons en compte la valeur de la fonction objectif, c'est-à-dire le coût total, et nous calculerons grâce à celle-ci l'écart relatif par rapport à la meilleure solution connue selon l'instance. Cet **écart à l'optimum**, exprimé en pourcentage, s'obtient avec la formule :
$$
Écart = \frac{résultat - BKS}{BKS} \times 100
$$
Pour mesurer la robustesse, chaque instance sera exécutée plusieurs fois. Nous analyserons ensuite la moyenne, l'écart-type et les extremums obtenus sur ces exécutions. En effet, de par la nature stochastique de notre méta-heuristique, on ne peut se contenter d'une seule exécution.

#### **2.3. MÉTHODOLOGIE D'ANALYSE**

L'analyse des données est nécessaire pour répondre aux questions de recherche. C'est elle qui nous permet d'évaluer les forces et faiblesses de notre algorithme. Nous utiliserons principalement des outils issus des statistiques descriptives et des représentations graphiques. Les courbes de convergence nous seront particulièrement utiles pour montrer l'évolution de l'optimum trouvé au fur et à mesure du temps ou des itérations. Les boîtes à moustaches permettront de mieux visualiser les distributions.

## **3. PLAN D'EXPÉRIENCE**

Pour calibrer les **facteurs** de notre méta-heuristique, c'est-à-dire les paramètres de l'algorithme qui influent sur l'intensité des mécanismes implémentés, nous pouvons naïvement faire varier un seul paramètre à la fois jusqu'à obtenir un bon résultat. Cependant, cette méthode est à la fois extrêmement coûteuse en temps de calcul et incapable de détecter les interactions entre les paramètres. Il est fréquent que l'effet d'un paramètre dépende de la valeur d'un autre. C'est pourquoi nous mettons en place un plan d'expérience, avec l'objectif de trouver une configuration de facteurs globalement robuste et performante.

#### **3.1. IDENTIFICATION DES FACTEURS ET DES NIVEAUX**

La première étape consiste à lister les paramètres de notre méta-heuristique qui semblent avoir un impact sur sa performance et définir pour chacune un niveau . Nous avons identifié quatre facteurs principaux, tous quantitatifs, qui régissent l'équilibre entre diversification et intensification, la vitesse d'adaptation et le critère d'acceptation des solutions.

Bien qu'il existe d'autres facteurs, nous nous sommes limités aux principaux, car le nombre d'expériences (d'exécutions) à mener est égale à $n^{k}$, avec $k$ le nombre de facteurs et $n$ le nombre de niveaux par facteur. Notre sélection des facteurs et de leurs niveaux s'appuie sur notre propre analyse préliminaire de leur influence potentielle et sur les pratiques courantes dans la [littérature scientifique **¹**](#1).

Nous avons :
- Le taux $\alpha$ de refroidissement, qui permet d'accepter des solutions de moins bonne qualité avec une probabilité qui diminue au fil du temps, et ce pour éviter les blocages, entre 0.85 et 0.99
- Le degré de destruction, qui contrôle le nombre de clients à retirer de la solution courante, entre 10% et 40%. Nous savons déjà qu'un faible degré favorise l'intensification tandis qu'un degré élevé favorise la diversification.
- Le regret pour l'insertion, qui définit le nombre de meilleures positions alternatives à considérer pour un client lors de son insertion, parmi 2, 3 ou 4.
- Le facteur $\rho$ de réduction pour l'adaptation des poids, qui contrôle la vitesse à laquelle les poids des opérateurs de destruction et de répartition s'adaptent en fonction de leur performance passée, entre 0.5 et 0.9.

Les autres paramètres, tels que la température initiale ou les scores d'attribution de récompense seront fixés à des valeurs standards pour ne pas complexifier excessivement l'analyse.

#### **3.2. CHOIX DU DESIGN EXPÉRIMENTAL**

#### **3.3. EXÉCUTION ET ANALYSE**

## **4. ANALYSE**

## **5. BIBLIOGRAPHIE**

<a id="1">1</a> — ALNS, ULM University (2014), section 5.4 : https://d-nb.info/1072464683/34